# Regression @ Training at delivery level, testing at hexagon level


In [1]:
import os
import sys
from pathlib import Path
from scipy import stats

# add the
ROOT = os.path.join(Path(os.getcwd()), "cargo-bike-analysis")

sys.path.append(str(ROOT))

from src.config import CargoBikeConfig, load_config
from src.osm_tags import build_tag_filter
from urban_tools.hex_pipeline import RouteHexHandler, TestTrainManager


import polars as pl
import geopolars as gpl
import geopandas as gpd
import pandas as pd
import numpy as np

## Load Config


In [2]:
config = load_config(ROOT + "/config" + "/paper.yaml")

In [3]:
h3_df = pd.concat(
    [gpd.read_parquet(city.h3_file).assign(city=city.name) for city in config.Cities],
    axis=0,
).query("is_city")

h3_df.head()

,geometry,is_city,city
region_id,,,
892a339a5afffff,"POLYGON ((-71.13572 42.23376, -71.13794 42.232...",True,"Boston, USA"
892a3066a3bffff,"POLYGON ((-71.08114 42.30902, -71.08337 42.308...",True,"Boston, USA"
892a302a567ffff,"POLYGON ((-70.82381 42.36269, -70.82604 42.361...",True,"Boston, USA"
892a3066e17ffff,"POLYGON ((-71.06072 42.33323, -71.06295 42.332...",True,"Boston, USA"
892a3066b3bffff,"POLYGON ((-71.06614 42.29023, -71.06837 42.289...",True,"Boston, USA"


In [4]:
## Load the Service Time Data

delivery_df = pl.concat(
    [
        pl.read_parquet(city.file).select(
            ["h3", pl.col(city.service_time_col).alias("service_time")]
        )
        for city in config.ServiceTime
    ],
)

# this does two things, one adds the city label and 2, it crops to the city limits
service_time_df = delivery_df.join(
    pl.DataFrame(h3_df.reset_index()[["region_id", "city"]]),
    left_on="h3",
    right_on="region_id",
    how="inner",
).with_columns(
    [
        pl.col("service_time").log().alias("service_time_log"),
    ]
)


service_time_df = service_time_df.with_columns(pl.count().over("h3").alias("h3_count"))

## Load the Embeddings


In [5]:
embedding_df = pl.read_parquet(config.GeoVex.embedding_file)

embedding_df = embedding_df.join(
    service_time_df, left_on="region_id", right_on="h3", how="inner"
)
embedding_df = pd.DataFrame(embedding_df, columns=embedding_df.columns)
embedding_df.head()

,0,1,2,3,4,5,6,7,8,9,...,45,46,47,48,49,region_id,service_time,city,service_time_log,h3_count
0,-5.32002,-5.493441,5.661098,20.907742,27.271231,-51.597836,-10.332759,45.391235,11.334351,10.848624,...,9.81983,5.172642,-7.319407,4.488636,20.39122,891fa441b83ffff,197.78692,"Brussels, Belgium",5.28719,201
1,-8.380403,-6.508181,18.301435,32.224045,28.388454,-35.504196,-18.33882,45.910313,19.495548,10.839149,...,15.916649,-1.365356,-8.613139,1.902637,28.177177,891fa441b53ffff,116.54679,"Brussels, Belgium",4.758293,187
2,-10.001373,-4.152925,16.126925,32.863831,27.293434,-32.102943,-19.849827,43.94511,20.050192,7.394312,...,14.05009,0.481978,-9.877087,-1.062505,28.223278,891fa441a63ffff,315.802888,"Brussels, Belgium",5.755118,10
3,-9.019328,-2.320791,13.435673,29.301117,26.384064,-32.886482,-17.608721,41.663998,17.434343,4.269286,...,11.727448,1.323634,-7.122827,-0.356202,25.736353,891fa441a6fffff,272.461272,"Brussels, Belgium",5.607496,10
4,-6.49675,-6.633764,18.607834,38.7845,26.904194,-21.650276,-21.75276,40.748585,23.255447,2.466128,...,10.815602,3.508712,-9.7584,-1.407086,28.712811,891fa44f4d7ffff,184.225706,"Brussels, Belgium",5.216162,24


## Regression Model


In [6]:
from typing_extensions import TypedDict
from typing import List
from typing import Union
from mapie.subsample import Subsample

from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import RobustScaler

from ngboost import NGBRegressor
from ngboost.learners import default_tree_learner, default_linear_learner
from ngboost.scores import CRPS, MLE
from ngboost.distns import LogNormal, Normal

import warnings
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import stats

from mapie.metrics import regression_coverage_score
from mapie.regression import MapieRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.gaussian_process import GaussianProcessRegressor

from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, r2_score
from sklearn.metrics import mean_pinball_loss
from scipy.stats import spearmanr
import CRPS.CRPS as pscore
import plotly.express as px
import plotly.graph_objects as go

warnings.filterwarnings("ignore")
alpha = 0.05

In [7]:
def split_test_train(
    df,
    feature_cols,
    target_col,
    train_regions: List[str] = None,
    test_regions: List[str] = None,
    test_size=0.2,
):
    unique_regions = df["region_id"].unique()
    # split the unique regions into train and test
    if train_regions is None and test_regions is None:
        train_regions, test_regions = train_test_split(
            unique_regions, test_size=test_size
        )

    train_df = df[df["region_id"].isin(train_regions)].copy()

    #     test_df = (
    #         df[df["region_id"].isin(test_regions)]
    #         .groupby("region_id")
    #         .agg({
    #             f'{target_col}': "median",
    #             **{col: "first" for col in feature_cols}
    #         })
    #         .reset_index()
    #         .copy()
    #     )

    return train_df


#     return train_df, test_df


def rejection_sampling(i, Y_dists, y_lb, y_ub):
    sample = Y_dists.sample(1)[0][i]
    #     count = 0
    while (sample < y_lb) or (sample > y_ub):
        sample = Y_dists.sample(1)[0][i]
    #         count+=1
    #         if count==200:
    #             return None
    return sample


def get_del_df(city, k=10):
    city_df = embedding_df[embedding_df["city"].str.contains(city)]
    #     city_df = embedding_df[embedding_df['city'].isin(cities)]
    return city_df[city_df["h3_count"] >= k]


def h3_aggregated_df(df):
    return (
        df.groupby("region_id")
        .agg({f"{target_col}": "median", **{col: "first" for col in feature_cols}})
        .reset_index()
        .copy()
    )


def get_empirical_pnts(alpha, del_h3_df):
    # 90th Percentile
    def lower_quantile(x):
        return x.quantile(alpha)

    # 90th Percentile
    def upper_quantile(x):
        return x.quantile(1 - alpha)

    del_h3_df.set_index("h3", inplace=True)
    h3_df = del_h3_df.groupby("h3").agg(
        {"service_time": [lower_quantile, "median", upper_quantile]}
    )
    h3_df.columns = list(map("_".join, h3_df.columns.values))

    # Collect Empirical points
    s = del_h3_df.groupby("h3").apply(lambda x: x["service_time"].values)
    empirical_pts_df = pd.merge(
        h3_df, s.rename("empirical_pts"), left_index=True, right_index=True
    )
    return empirical_pts_df


def get_test_scores(df):
    (
        median_r2s,
        median_rmses,
        median_KS_scores,
        median_pinball_scores,
        CRPS_median_RS_scores,
    ) = ([], [], [], [], [])
    y_pred_lb, y_pred_ub = (
        df["service_time_lower_quantile_rj_pred"],
        df["service_time_upper_quantile_rj_pred"],
    )

    cov_score = np.round(
        regression_coverage_score(
            df["service_time_median_rj_pred"].values, y_pred_lb, y_pred_ub
        ),
        3,
    )
    avg_width = np.round((y_pred_ub - y_pred_lb).mean(), 3)
    upp_quan_r2 = np.round(
        r2_score(df["service_time_upper_quantile_true"].values, y_pred_ub), 3
    )
    upp_quan_rmse = np.round(
        mean_squared_error(
            df["service_time_upper_quantile_true"].values, y_pred_ub, squared=False
        ),
        3,
    )

    for hx in df.itertuples():
        temp, pemp = getattr(hx, "empirical_pts_true"), getattr(
            hx, "empirical_pts_rj_pred"
        )
        median_r2s.append(np.round(r2_score(temp, pemp), 3))
        median_rmses.append(np.round(mean_squared_error(temp, pemp, squared=False), 3))
        median_KS_scores.append(stats.ks_2samp(temp, pemp))
        median_pinball_scores.append(mean_pinball_loss(temp, pemp))
    #         CRPS_median_RS_scores.append(pscore(pemp,getattr(hx,'service_time_median_rj_pred')).compute())

    return (
        cov_score,
        avg_width,
        upp_quan_r2,
        upp_quan_rmse,
        np.mean(np.array(median_r2s)),
        np.mean(np.array(median_rmses)),
        np.mean(np.array(median_KS_scores)),
        np.mean(np.array(median_pinball_scores)),
    )


#     return cov_score, avg_width, upp_quan_r2, upp_quan_rmse, np.mean(np.array(median_r2s)), np.mean(np.array(median_rmses)), np.mean(np.array(median_KS_scores)), np.mean(np.array(median_pinball_scores)), np.mean(CRPS_median_RS_scores, axis=0).tolist()

In [8]:
# # Nice test
# df=get_del_df('Boston, USA', 20)
# df = df.rename(columns={'region_id':'h3'})
# f=get_empirical_pnts(alpha, df)
# # g=f.copy()
# f.head()

In [9]:
# Specify the feature and target columns
feature_cols, target_col = list(map(str, range(50))), "service_time_log"


# ------------------- Loop over cities and cross validation folds -------------------
def retrieve_the_cv_splits(cities, n_splits):
    # Cross Validation Folds
    kf = KFold(n_splits=n_splits)

    city_sets = {}
    city_sets["train"] = {}
    #     city_sets['test'] = {}
    for i in range(n_splits):
        city_sets["train"][f"split_{i}"] = pd.DataFrame()

    for city in cities:
        #         city_sets['test'][city] = {}

        # get the city dataframe
        city_df = get_del_df(city, 20)

        # get the unique regions
        unique_regions = city_df["region_id"].unique().tolist()

        for i, (train_index, val_index) in enumerate(kf.split(unique_regions)):
            train_regions = [unique_regions[i] for i in train_index]
            val_regions = [unique_regions[i] for i in val_index]
            train_df = split_test_train(
                city_df,
                feature_cols,
                target_col,
                train_regions=train_regions,
                test_regions=val_regions,
            )

            if not city_sets["train"][f"split_{i}"].empty:
                city_sets["train"][f"split_{i}"] = city_sets["train"][
                    f"split_{i}"
                ].append(train_df)
            else:
                city_sets["train"][f"split_{i}"] = train_df

    #             city_sets['test'][city][f'split_{i}'] = test_df

    return city_sets


def get_predictions_entirety(ngb, mapie, df, alpha=alpha):
    region_ids = df["region_id"].to_list()
    _, tmp = mapie.predict(df[feature_cols].to_numpy(), alpha=[alpha, 0.5])
    Y_dists = ngb.pred_dist(df[feature_cols].to_numpy())
    y_pred, y_pis = tmp[:, 1, 0].ravel(), tmp[:, :, 1]
    lower_quan = y_pis[:, 0].ravel()
    upper_quan = y_pis[:, 1].ravel()

    # Go to the hexagonal level for predictions on quantiles here
    # 1. First get true empirical points aggregated on hexes
    # Rejection sampling
    np.random.seed(42)
    dlvry_rej_sample_service_time = [
        np.exp(rejection_sampling(i, Y_dists, lower_quan[i], upper_quan[i]))
        for i in range(y_pis.shape[0])
    ]
    df_with_h3_true_samples = pd.DataFrame(
        {"h3": region_ids, "service_time": np.exp(df[target_col].astype(float))}
    )
    hex_empirical = get_empirical_pnts(alpha, df_with_h3_true_samples)

    # 2. Now get rejection sampled predicted points aggregated on hexes
    df_with_h3_rj_samples = pd.DataFrame(
        {"h3": region_ids, "service_time": dlvry_rej_sample_service_time}
    )
    hex_pred_rj_samples_agg = get_empirical_pnts(alpha, df_with_h3_rj_samples)

    # 3. Get the empirical points in one dataset
    joint_h3_true_rj_pred_df = pd.merge(
        hex_empirical,
        hex_pred_rj_samples_agg,
        left_index=True,
        right_index=True,
        suffixes=("_true", "_rj_pred"),
    )
    return joint_h3_true_rj_pred_df


def get_preds_from_joint_model_city_wise(cities, pred_cities, alpha, n_splits):
    scores = {}
    predictions_df = {}
    # Initialize dictionaries
    for city in cities:
        scores[city] = {}

    city_sets = retrieve_the_cv_splits(cities, n_splits)

    for pred_city in pred_cities:
        #         cov_scores, avg_widths, median_r2s, median_rmses, upp_quan_r2s = [],[],[],[],[]
        #         upp_quan_rmses, median_KS_scores, median_pinball_scores = [],[],[]
        #         upp_quan_rmses, median_KS_scores, median_pinball_scores, CRPS_median_RS_scores = [],[],[],[]

        for i in range(n_splits):
            train_df = city_sets["train"][f"split_{i}"]
            # get the city dataframe
            pred_city_df = get_del_df(pred_city, 20)
            #             test_df = city_sets['test'][pred_city][f'split_{i}']
            #             test_df = pred_city_df.groupby("region_id").agg({
            #                 f'{target_col}': "median",
            #                 **{col: "first" for col in feature_cols}
            #             }).reset_index().copy()

            X_train, X_cal, y_train, y_cal = train_test_split(
                train_df[feature_cols].to_numpy(),
                train_df[target_col].to_numpy(),
                test_size=0.2,
                random_state=23,
            )

            ngb = NGBRegressor(
                n_estimators=1,
                learning_rate=1e-3,
                Dist=Normal,
                Base=default_tree_learner,
                natural_gradient=True,
                Score=MLE,
                verbose=False,
            )
            mapie = MapieRegressor(ngb).fit(X_cal, y_cal)
            mapie.fit(X_train, y_train)
            ngb.fit(X_train, y_train)

            predictions_df[pred_city] = get_predictions_entirety(
                ngb, mapie, pred_city_df, alpha
            )

    return predictions_df

## Joint Van Model


In [10]:
van_cities = ["Boston, USA", "Seattle, USA", "Austin, USA", "Chicago, USA"]
bike_cities = ["London, UK", "Brussels, Belgium"]

predictions_df = {}

predictions_df["vans"] = get_preds_from_joint_model_city_wise(
    bike_cities, van_cities, alpha=alpha, n_splits=2
)
predictions_df["bikes"] = get_preds_from_joint_model_city_wise(
    van_cities, bike_cities, alpha=alpha, n_splits=2
)

# # Get predictions
# van_scores = get_preds_from_joint_model_city_wise(van_cities, van_cities, alpha=alpha, n_splits=5)

import json, pickle

# Path to the saved JSON file
# score_file_path = f"{os.environ['HOME']}/CQR_Experiments/Van_joint_model_CV_KS_SCORES_NGB.json"
pred_file_path = (
    f"{os.environ['HOME']}/CQR_Experiments/DF_joint_model_CV_KS_predictions_NGB.pickle"
)

# # the json file where the output must be stored
# out_file = open(score_file_path, "w")

# json.dump(van_scores, out_file, indent = 6)
# out_file.close()

# Store data (serialize)
with open(pred_file_path, "wb") as handle:
    pickle.dump(predictions_df, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Load data (deserialize)
with open(pred_file_path, "rb") as handle:
    predictions_df = pickle.load(handle)

In [11]:
predictions_df["vans"]["Boston, USA"].head()

,service_time_lower_quantile_true,service_time_median_true,service_time_upper_quantile_true,empirical_pts_true,service_time_lower_quantile_rj_pred,service_time_median_rj_pred,service_time_upper_quantile_rj_pred,empirical_pts_rj_pred
h3,,,,,,,,
892a30640b7ffff,20.000,153.5,418.710,"[156.99999999999994, 110.99999999999997, 218.1...",103.288167,139.410674,192.176375,"[113.09576661431475, 126.87290342967748, 146.3..."
892a3064103ffff,40.150,147.5,517.090,"[464.09999999999997, 129.99999999999997, 85.00...",96.385137,128.203761,196.521651,"[113.62879843518037, 108.19469882470183, 198.6..."
892a3064107ffff,40.000,108.0,535.000,"[654.5000000000002, 40.0, 70.00000000000003, 2...",97.091020,134.381308,179.534020,"[103.28654508906489, 126.36018729890343, 149.7..."
892a306410bffff,53.595,106.0,474.600,"[149.99999999999997, 85.00000000000001, 74.999...",94.948332,127.603993,190.919405,"[95.24575520659074, 118.38285949150924, 166.96..."
892a306410fffff,32.750,94.0,490.275,"[104.99999999999997, 175.99999999999991, 214.9...",102.444977,150.543405,187.502256,"[122.04071667712383, 100.01081525871068, 129.1..."
